In [1]:
import torch
import torch.nn as nn
from torchvision import models


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("GPU:", torch.cuda.get_device_name(0))


Using device: cuda
GPU: NVIDIA GeForce RTX 3060


In [3]:
model = models.efficientnet_b6(
    weights=models.EfficientNet_B6_Weights.IMAGENET1K_V1
)


In [4]:
model = model.to(device)


In [5]:
print(model.classifier)


Sequential(
  (0): Dropout(p=0.5, inplace=True)
  (1): Linear(in_features=2304, out_features=1000, bias=True)
)


In [7]:
num_classes = 11


In [8]:
print(model.classifier)


Sequential(
  (0): Dropout(p=0.5, inplace=True)
  (1): Linear(in_features=2304, out_features=1000, bias=True)
)


In [9]:
model.classifier[1] = nn.Linear(
    in_features=model.classifier[1].in_features,
    out_features=num_classes
)


In [10]:
model = model.to(device)


In [11]:
print(model.classifier)


Sequential(
  (0): Dropout(p=0.5, inplace=True)
  (1): Linear(in_features=2304, out_features=11, bias=True)
)


In [ ]:
for param in model.features.parameters():
    param.requires_grad = False
#freezling  backbone parameters  , not changing image-understanding layers

In [12]:
trainable_params = sum(p.requires_grad for p in model.parameters())
total_params = sum(1 for _ in model.parameters())

print(f"Trainable parameters: {trainable_params}")
print(f"Total parameters: {total_params}")


Trainable parameters: 584
Total parameters: 584


In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


image transformation

In [14]:
IMG_SIZE = 600

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [15]:
DATASET_PATH = r"C:\Users\ADMIN\Documents\AM Project\Pandora_18k - AM"

full_dataset = datasets.ImageFolder(
    root=DATASET_PATH,
    transform=transform
)


In [16]:
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size]
)


In [17]:
BATCH_SIZE = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)


In [18]:
print("Number of classes:", len(full_dataset.classes))
print("Class names:", full_dataset.classes)

images, labels = next(iter(train_loader))
print("Batch image shape:", images.shape)
print("Batch label shape:", labels.shape)


Number of classes: 11
Class names: ['02_Early_Renaissance', '03_Northern_Renaissance', '04_High_Renaissance', '05_Baroque', '08_Realism', '09_Impressionism', '10_Post_Impressionism', '11_Expressionism', '13_Fauvism', '14_Cubism', '16_AbstractArt']
Batch image shape: torch.Size([8, 3, 600, 600])
Batch label shape: torch.Size([8])


In [19]:
criterion = nn.CrossEntropyLoss()


In [20]:
optimizer = torch.optim.Adam(
    model.classifier.parameters(),
    lr=1e-4
)


In [21]:
EPOCHS = 100


In [22]:
patience = 10
best_val_loss = float("inf")
epochs_without_improvement = 0


In [23]:
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return (preds == labels).sum().item()


In [24]:
from tqdm import tqdm

history = {
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": []
}

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("-" * 30)

    # =====================
    # Training phase
    # =====================
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_train = 0

    for images, labels in tqdm(train_loader, desc="Training"):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_corrects += calculate_accuracy(outputs, labels)
        total_train += labels.size(0)

    epoch_train_loss = running_loss / total_train
    epoch_train_acc = running_corrects / total_train

    # =====================
    # Validation phase
    # =====================
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            val_corrects += calculate_accuracy(outputs, labels)
            total_val += labels.size(0)

    epoch_val_loss = val_loss / total_val
    epoch_val_acc = val_corrects / total_val

    history["train_loss"].append(epoch_train_loss)
    history["val_loss"].append(epoch_val_loss)
    history["train_acc"].append(epoch_train_acc)
    history["val_acc"].append(epoch_val_acc)

    print(f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}")
    print(f"Val   Loss: {epoch_val_loss:.4f} | Val   Acc: {epoch_val_acc:.4f}")

    # =====================
    # Early stopping logic
    # =====================
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), "best_efficientnet_b6.pth")
        print("✅ Validation loss improved. Model saved.")
    else:
        epochs_without_improvement += 1
        print(f"⚠️ No improvement for {epochs_without_improvement} epoch(s).")

    if epochs_without_improvement >= patience:
        print("🛑 Early stopping triggered.")
        break



Epoch 1/100
------------------------------


Validation: 100%|██████████| 276/276 [23:39<00:00,  5.14s/it]


Train Loss: 2.1333 | Train Acc: 0.3266
Val   Loss: 1.9756 | Val   Acc: 0.3679
✅ Validation loss improved. Model saved.

Epoch 2/100
------------------------------


Validation: 100%|██████████| 276/276 [23:39<00:00,  5.14s/it]


Train Loss: 1.8205 | Train Acc: 0.4439
Val   Loss: 1.7618 | Val   Acc: 0.4237
✅ Validation loss improved. Model saved.

Epoch 3/100
------------------------------


...

Train Loss: 1.2932 | Train Acc: 0.5656
Val   Loss: 1.2530 | Val   Acc: 0.5863
✅ Validation loss improved. Model saved.

Epoch 13/100
------------------------------


Validation: 100%|██████████| 276/276 [29:49<00:00,  6.48s/it]


Train Loss: 1.2755 | Train Acc: 0.5623
Val   Loss: 1.2494 | Val   Acc: 0.5822
✅ Validation loss improved. Model saved.

Epoch 14/100
------------------------------


Training:   0%|          | 4/1104 [02:43<12:28:56, 40.85s/it]


KeyboardInterrupt: 

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
from torchvision import models
import torch.nn as nn

num_classes = 11  # make sure this matches your dataset

# Load pretrained EfficientNet-B6
model = models.efficientnet_b6(weights="IMAGENET1K_V1")

# Replace classifier
model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    num_classes
)

# Move model to device
model = model.to(device)

print("✅ EfficientNet-B6 architecture created")


✅ EfficientNet-B6 architecture created


In [5]:
# Load best saved B6 model weights
model.load_state_dict(
    torch.load("best_efficientnet_b6.pth", map_location=device)
)

model = model.to(device)

print("✅ Best EfficientNet-B6 checkpoint loaded successfully")


✅ Best EfficientNet-B6 checkpoint loaded successfully


In [6]:
import torch
torch.backends.cudnn.benchmark = True

print("✅ cuDNN benchmark enabled")


✅ cuDNN benchmark enabled


In [7]:
IMG_SIZE = 528
print("✅ Image size set to", IMG_SIZE)


✅ Image size set to 528


In [8]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)

print("✅ Optimized DataLoaders created")


NameError: name 'train_dataset' is not defined

In [9]:
from torchvision import datasets, transforms

IMG_SIZE = 528  # B6 correct size

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("✅ Transforms redefined")


✅ Transforms redefined


In [10]:
DATASET_PATH = r"C:\Users\ADMIN\Documents\AM Project\Pandora_18k - AM"

full_dataset = datasets.ImageFolder(
    root=DATASET_PATH,
    transform=transform
)

print("✅ Full dataset loaded")
print("Number of classes:", len(full_dataset.classes))


✅ Full dataset loaded
Number of classes: 11


In [11]:
from torch.utils.data import random_split

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size]
)

print("✅ Train/Validation split recreated")
print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))


✅ Train/Validation split recreated
Train samples: 8828
Validation samples: 2207


In [12]:
from torch.utils.data import DataLoader

BATCH_SIZE = 8  # keep same for fair comparison

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)

print("✅ Optimized DataLoaders created")


✅ Optimized DataLoaders created


In [13]:
# Fetch one batch from train loader
images, labels = next(iter(train_loader))

print("Batch image shape:", images.shape)
print("Batch label shape:", labels.shape)
print("Image dtype:", images.dtype)
print("Label dtype:", labels.dtype)


Batch image shape: torch.Size([8, 3, 528, 528])
Batch label shape: torch.Size([8])
Image dtype: torch.float32
Label dtype: torch.int64


In [14]:
# Maximum epochs (same as before)
EPOCHS = 100

# Early stopping settings (same as before)
patience = 10
epochs_without_improvement = 0

# IMPORTANT: reset best_val_loss so early stopping works correctly
best_val_loss = float("inf")

print("✅ Training control variables reset and ready")


✅ Training control variables reset and ready


In [15]:
model.train()
print("✅ Model set to training mode")


✅ Model set to training mode


In [17]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
print("✅ Loss function defined")


✅ Loss function defined


In [18]:
optimizer = torch.optim.Adam(
    model.classifier.parameters(),
    lr=1e-4
)

print("✅ Optimizer redefined")


✅ Optimizer redefined


In [19]:
from tqdm import tqdm

history = {
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": []
}

def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return (preds == labels).sum().item()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("-" * 30)

    # =====================
    # Training
    # =====================
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_train = 0

    for images, labels in tqdm(train_loader, desc="Training"):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_corrects += calculate_accuracy(outputs, labels)
        total_train += labels.size(0)

    epoch_train_loss = running_loss / total_train
    epoch_train_acc = running_corrects / total_train

    # =====================
    # Validation (every 2 epochs for speed)
    # =====================
    if epoch % 2 == 0:
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                val_corrects += calculate_accuracy(outputs, labels)
                total_val += labels.size(0)

        epoch_val_loss = val_loss / total_val
        epoch_val_acc = val_corrects / total_val

        history["val_loss"].append(epoch_val_loss)
        history["val_acc"].append(epoch_val_acc)

        print(f"Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}")

        # Early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            epochs_without_improvement = 0
            torch.save(model.state_dict(), "best_efficientnet_b6.pth")
            print("✅ Validation improved — model saved")
        else:
            epochs_without_improvement += 1
            print(f"⚠️ No improvement for {epochs_without_improvement} epoch(s)")

        if epochs_without_improvement >= patience:
            print("🛑 Early stopping triggered")
            break

    history["train_loss"].append(epoch_train_loss)
    history["train_acc"].append(epoch_train_acc)

    print(f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}")



Epoch 1/100
------------------------------


Training:  12%|█▏        | 132/1104 [32:44<4:01:04, 14.88s/it]


KeyboardInterrupt: 